## Working with DGGAL in Vgrid DGGS

[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.vgrid.vn/lab/index.html?path=vgrid/08_dggal.ipynb)
[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/opengeoshub/vgrid/blob/main/docs/notebooks/08_dggal.ipynb)
[![image](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/opengeoshub/vgrid/HEAD?filepath=docs/notebooks/08_dggal.ipynb)
[![image](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/opengeoshub/vgrid/blob/main/docs/notebooks/08_dggal.ipynb)
[![image](https://jupyterlite.rtfd.io/en/latest/_static/badge.svg)](https://demo.gishub.vn/lab/index.html?path=notebooks/vgrid/08_dggal.ipynb)

Full Vgrid DGGS documentation is available at [vgrid document](https://vgrid.gishub.vn).

To work with Vgrid DGGS directly in GeoPandas and Pandas, please use [vgridpandas](https://pypi.org/project/vgridpandas/). Full Vgridpandas DGGS documentation is available at [vgridpandas document](https://vgridpandas.gishub.vn).

To work with Vgrid DGGS in QGIS, install the [Vgrid Plugin](https://plugins.qgis.org/plugins/vgridtools/).

To visualize DGGS in Maplibre GL JS, try the [vgrid-maplibre](https://www.npmjs.com/package/vgrid-maplibre) library.

For an interactive demo, visit the [Vgrid Homepage](https://vgrid.vn).

### Install vgrid
Uncomment the following line to install [vgrid](https://pypi.org/project/vgrid/).

In [ ]:
# %pip install vgrid --upgrade

### latlon2dggal

In [ ]:
from vgrid.conversion.latlon2dggs import latlon2dggal

dggs_type = "isea9r"  # choose one from ['gnosis','isea3h','isea9r','ivea3h',
# 'ivea9r','rtea3h','rtea9r','rhealpix']
resolution = 10
dggal_id = latlon2dggal(dggs_type, 10.775276, 106.706797, resolution)
dggal_id

### DGGAL to Polygon

In [ ]:
from vgrid.conversion.dggs2geo.dggal2geo import dggal2geo

dggal_geo = dggal2geo(dggs_type, dggal_id)
dggal_geo

### DGGAL to GeoJSON

In [ ]:
from vgrid.conversion.dggs2geo.dggal2geo import dggal2geojson
import json

dggal_geojson = dggal2geojson(dggs_type, dggal_id)
print(json.dumps(dggal_geojson))

### Vector to DGGAL

In [ ]:
from vgrid.conversion.vector2dggs.vector2dggal import vector2dggal

file_path = "https://raw.githubusercontent.com/opengeoshub/vopendata/main/shape/polygon2.geojson"
dggs_type = "isea9r"  # "gnosis", "isea3h", "isea9r", "ivea3h", "ivea9r", "rtea3h", "rtea9r", "rhealpix"
resolution = 10
vector_to_dggal = vector2dggal(
    dggs_type=dggs_type,
    vector_data=file_path,
    resolution=resolution,
    compact=False,
    predicate="intersects",
    output_format="gpd",
)
# Visualize the output
vector_to_dggal.plot(edgecolor="white")

### DGGAL Compact

In [ ]:
from vgrid.conversion.dggscompact.dggalcompact import dggalcompact

dggal_compacted = dggalcompact(dggs_type, vector_to_dggal, output_format="gpd")
dggal_compacted.plot(edgecolor="white")

### DGGAL Expand

In [ ]:
from vgrid.conversion.dggscompact.dggalcompact import dggalexpand

# dggs_type = 'isea3h' # 'gnosis','isea3h','isea9r','ivea3h','ivea9r','rtea3h','rtea9r','rhealpix'
dggal_expanded = dggalexpand(
    dggs_type, vector_to_dggal, resolution=11, output_format="gpd"
)
# dggal_expanded
dggal_expanded.plot(edgecolor="white")

### DGGAL Binning

In [ ]:
from vgrid.binning.dggalbin import dggalbin

dggs_type = "isea9r"  # choose one from ['gnosis','isea3h','isea9r','ivea3h',
# 'ivea9r','rtea3h','rtea9r','rhealpix']
file_path = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/csv/housing.csv"
)
stats = "max"
dggal_bin = dggalbin(
    dggs_type,
    file_path,
    resolution=6,
    stats=stats,
    numeric_field="median_house_value",
    # category="category",
    output_format="gpd",
)
# dggal_bin.head()
dggal_bin.plot(
    column=stats,  # numeric column to base the colors on
    cmap="Spectral_r",  # color scheme (matplotlib colormap)
    legend=True,
    linewidth=0.2,  # boundary width (optional)
)

### Raster to DGGAL

#### Download and open raster

In [ ]:
from vgrid.utils.io import download_file
import rasterio
from rasterio.plot import show

raster_url = (
    "https://raw.githubusercontent.com/opengeoshub/vopendata/main/raster/rgb.tif"
)
raster_file = download_file(raster_url)
src = rasterio.open(raster_file, "r")
print(src.meta)
show(src)

#### Convert raster to DGGAL

In [ ]:
# %pip install folium

In [ ]:
from vgrid.conversion.raster2dggs.raster2dggal import raster2dggal

dggs_type = "isea3h"  # 'gnosis','isea3h','isea9r','ivea3h','ivea9r','rtea3h','rtea9r','rhealpix'
raster_to_dggal = raster2dggal(dggs_type, raster_file, output_format="gpd")

# Visualize the output
import folium

m = folium.Map(tiles="CartoDB positron", max_zoom=28)

dggal_layer = folium.GeoJson(
    raster_to_dggal,
    style_function=lambda x: {
        "fillColor": f"rgb({x['properties']['band_1']}, {x['properties']['band_2']}, {x['properties']['band_3']})",
        "fillOpacity": 1,
        "color": "black",
        "weight": 0.8,
    },
    popup=folium.GeoJsonPopup(
        fields=[f"dggal_{dggs_type}", "band_1", "band_2", "band_3"],
        aliases=["Zone ID", "Band 1", "Band 2", "Band 3"],
        style="""
            background-color: white;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
    ),
).add_to(m)

m.fit_bounds(dggal_layer.get_bounds())

# Display the map
m

### DGGAL Generator

In [ ]:
from vgrid.generator.dggalgen import dggalgen

dggs_type = "rhealpix"  # "gnosis", "isea3h", "isea9r", "ivea3h", "ivea9r", "rtea3h", "rtea9r", "rhealpix"
resolution = 1
bbox = (
    106.699007,
    10.762811,
    106.717674,
    10.778649,
)  # (min_lon, min_lat, max_lon, max_lat)
dggal_grid = dggalgen(
    dggs_type=dggs_type,
    resolution=resolution,
    # bbox=bbox,
    # compact=True,
    output_format="gpd",
)
dggal_grid
dggal_grid.plot(edgecolor="white")

### DGGAL Inspect

In [ ]:
from vgrid.stats.dggalstats import dggalinspect

dggs_type = "isea3h"  # choose one from ['gnosis','isea3h','isea9r','ivea3h',
# 'ivea9r','rtea3h','rtea9r','rhealpix']
resolution = 6
dggal_inspect = dggalinspect(dggs_type, resolution)
dggal_inspect.head()

### DGGAL Normalized Area Histogram

In [ ]:
from vgrid.stats.dggalstats import dggal_norm_area_hist

dggal_norm_area_hist(dggs_type,dggal_inspect)

### Distribution of DGGAL Area Distortions

In [ ]:
from vgrid.stats.dggalstats import dggal_norm_area

dggal_norm_area(dggs_type, dggal_inspect, crs="proj=moll")

### DGGAL IPQ Compactness Histogram

Isoperimetric Inequality (IPQ) Compactness (suggested by [Osserman, 1978](https://sites.math.washington.edu/~toro/Courses/20-21/MSF/osserman.pdf)):

$$C_{IPQ} = \frac{4 \pi A}{p^2}$$
The range of the IPQ compactness metric is [0,1]. 

A circle represents the maximum compactness with a value of 1. 

As shapes become more irregular or elongated, their compactness decreases toward 0.

In [ ]:
from vgrid.stats.dggalstats import dggal_compactness_ipq_hist

dggal_compactness_ipq_hist(dggs_type, dggal_inspect)

### Distribution of DGGAL IPQ Compactness

In [ ]:
from vgrid.stats.dggalstats import dggal_compactness_ipq

dggal_compactness_ipq(dggs_type,dggal_inspect, crs="proj=moll")

DGGAL Convex hull Compactness Histogram:

$$C_{CVH} = \frac{A}{A_{CVH}}$$


The range of the Convex hull compactness metric is [0,1]. 

A circle represents the maximum compactness with a value of 1. 

As shapes become more irregular or elongated, their compactness decreases toward 0.

In [ ]:
from vgrid.stats.dggalstats import dggal_compactness_cvh_hist
dggal_compactness_cvh_hist(dggs_type, dggal_inspect)

### Distribution of DGGAL Convex hull Compactness

In [ ]:
from vgrid.stats.dggalstats import dggal_compactness_cvh
dggal_compactness_cvh(dggs_type, dggal_inspect)

### DGGAL Statistics

Characteristic Length Scale (CLS - suggested by Ralph Kahn): the diameter of a spherical cap of the same cell's area

In [ ]:
from vgrid.stats.dggalstats import dggalstats

dggs_type = "isea9r"  # choose one from ['gnosis','isea3h','isea9r','ivea3h',
# 'ivea9r','rtea3h','rtea9r','rhealpix']
dggal_stats = dggalstats(dggs_type, unit="km")
dggal_stats